# Summary

The target of the project is to create a supervised model with sequencial data deep learning model, to predict if a tweet indicates a real disaster. This is useful for some organizations to actively monitoring tweets for deteting disaster in near real time. 

In [1]:
!pip install translators

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 45.6 MB/s eta 0:00:0000:0100:01
  Created wheel for PyExecJS: filename=PyExecJS-1.5.1-py3-none-any.whl size=14578 sha256=0b023743ffe9776ee78f802a52ad2130e5a7d48390d3fcaf58253540364fba89
  Stored in directory: /root/.cache/pip/wheels/9d/91/30/28e6da53d4f44dc445349b2ffad581968447e4cbc9dd7991b8
Successfully built PyExecJS
  Attempting uninstall: cryptography
    Found existing installation: cryptography 41.0.7
    Uninstalling cryptography-41.0.7:
      Successfully uninstalled cryptography-41.0.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 23.3.0 requires cryptography<42,>=41.0.5, but you have crypto

In [5]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import string
import pickle
import time

import translators as ts
from translators.server import tss
import copy

# Import tqdm for progress bar
from tqdm.notebook import tqdm


In [6]:
test_text = "13,000 people receive #wildfires evacuation orders in California"

In [7]:


def test_translator(text, translator, to_language="cn"):
    print("Original text:", text)
    try:
        print("Test translator:", translator)
        to_text = ts.translate_text(text, translator=translator, from_language='en', to_language=to_language, timeout=5)
        print("to_language:", to_text)
        back_en_text = ts.translate_text(to_text, translator=translator, from_language=to_language, to_language='en', timeout=5)
        print("back en:", back_en_text)
        if to_text is None or len(to_text.strip())==0 or back_en_text is None or len(back_en_text.strip())==0:
            print("Add", translator, "to failed list because of emppt result")
            failed_translators.append(translator)
    except Exception as e:
        print("failed because of e:", e)
        
test_translator(" haha its alrightbut more than twice is just stupid  hes traumatised ", "reverso")

Original text:  haha its alrightbut more than twice is just stupid  hes traumatised 
Test translator: reverso
to_language: 哈其两次以上的alrightbut只是愚蠢的hes traumatised
back en: Hagrid more than twice alrightbut is just stupid hes traumatised


In [8]:
def test_translators(text, translators, to_language="cn"):
    
    failed_translators = []
    
    for translator in translators:
        print("=================")
        try:
            print("Test translator:", translator)
            to_text = ts.translate_text(text, translator=translator, from_language='en', to_language=to_language, timeout=5)
            print("to_language:", to_text)
            back_en_text = ts.translate_text(to_text, translator=translator, from_language=to_language, to_language='en', timeout=5)
            print("back en:", back_en_text)
            if to_text is None or len(to_text.strip())==0 or back_en_text is None or len(back_en_text.strip())==0:
                print("Add", translator, "to failed list because of emppt result")
                failed_translators.append(translator)
        except Exception as e:
            print("Add", translator, "to failed list because of e:", e)
            failed_translators.append(translator)
    
    return failed_translators


#translators = copy.deepcopy(tss.translators_pool)

#failed_translators_cn = test_translators(test_text, cn_translators, to_language="cn")
#failed_translators_fr = test_translators(test_text, translators, to_language="fr")

In [9]:


cn_translators = [
    'alibaba', 'baidu', 'bing', 'caiyun', 'cloudTranslation', 'google', 'hujiang', 'iciba','iflyrec', 'itranslate', 
    'lingvanex', 'modernMt', 'papago', 'qqTranSmart', 'reverso', 'sogou', 'sysTran', 'translateCom', 'youdao'
]

de_translators = [
    'alibaba', 'argos', 'baidu', 'bing', 'cloudTranslation', 'google', 'hujiang', 'iciba', 'itranslate', 'languageWire',
    'modernMt', 'papago', 'qqFanyi', 'qqTranSmart', 'reverso', 'sogou', 'translateCom'
]

fr_translators = [
    'alibaba', 'argos', 'bing', 'cloudTranslation', 'elia', 'google', 'hujiang', 'iciba', 'languageWire', 'modernMt',
    'papago', 'qqFanyi', 'qqTranSmart', 'reverso', 'sogou', 'translateCom'
]

supported_lang_translators = [
    ("cn", cn_translators),
    ("de", de_translators),
    ("fr", fr_translators),
]

def random_pick_lang_translator():
    (lang, translators) = random.choice(supported_lang_translators)
    translator = random.choice(translators)
    return lang, translator, translators

lang, translator, translators = random_pick_lang_translator()
print(lang, translator, translators)
#translators

de translateCom ['alibaba', 'argos', 'baidu', 'bing', 'cloudTranslation', 'google', 'hujiang', 'iciba', 'itranslate', 'languageWire', 'modernMt', 'papago', 'qqFanyi', 'qqTranSmart', 'reverso', 'sogou', 'translateCom']


In [15]:
def translate_and_back(text, verbose=False, retry_limit=10, sleep_time=0.5):
    
    if verbose: print("original:", text)
    
    success = False
    retry=0
    while not success:
        retry += 1
        
        if retry > retry_limit:
            print("Retry limit reached:", retry)
            return None
        
        try:
            lang, translator, translators = random_pick_lang_translator()
            if verbose: 
                print("lang:", lang, ", translator:", translator)
                
            to_text = ts.translate_text(text, translator=translator, from_language="en", to_language=lang, timeout=5)
            # to avoid rate limit
            time.sleep(sleep_time)
            back_en_text = ts.translate_text(to_text, translator=translator, from_language=lang, to_language='en', timeout=5)
            
            if verbose: 
                print("to_language:", to_text)
                print("back en:", back_en_text)
            
            
            if back_en_text is not None or len(back_en_text.strip())>0:
                success = True
        except Exception as e:
            if verbose: 
                print("Removed", translator, " from", lang, " because of:", e)
                print("--------")
                translators.remove(translator)

    return back_en_text

translate_and_back("13,000 people receive #wildfires evacuation orders in California ", verbose=True)
print()
#print(random.choice(translators))

original: 13,000 people receive #wildfires evacuation orders in California 
lang: de , translator: baidu
to_language: 13.000 Menschen erhalten #Waldbrände Evakuierungsbefehle in Kalifornien
back en: 13,000 people receive #forest fires evacuation orders in California



# Load data

In [12]:
#DATA_ROOT = "/kaggle/input/nlp-getting-started"
#train_df=pd.read_csv(os.path.join(DATA_ROOT,'train.csv'))
train_df=pickle.load(open("/kaggle/input/nlp-disastertweets-1-eda-cleaning/train_cleaned_df.pkl", "rb"))
print("train data shape:", train_df.shape)

train data shape: (7613, 6)


In [14]:
# print some samples
for text in train_df.sample(n=5)["text"].values:
    translate_and_back(text, verbose=True)
    print("-------")

original: 22Beyonce Is my pick for  Fan Army Beyonce hive 
lang: de , translator: reverso
to_language: 22Beyonce Ist meine Wahl für Fan Army Beyonce hive
back en: 22Beyonce Is My Choice For Fan Army Beyonce hive
-------
original: Big Brother ouch Clelliyou could almost hear their hopes and dreams being crushed 
lang: de , translator: iciba
to_language: Sie können fast hören, wie ihre Hoffnungen und Träume zerbrochen sind
back en: You can almost hear their hopes and dreams dashed
-------
original: The majority of those killed were civilians on the ground after the jet first bombed the citys main street then dramatically plummeted
lang: fr , translator: qqFanyi
to_language: La majorité des personnes tuées étaient des civils sur le terrain après que l'avion ait d'abord bombardé la rue principale de la ville, puis s'est considérablement effondré.
back en: The majority of those killed were civilians on the ground after the plane first bombed the city's main street and then collapsed conside

## Translate and back

In [18]:
translated_text_list = []

for idx, text in enumerate(train_df["text"].values):
    if idx%100 == 0:
        print(idx, "/", len(train_df))
        
    trans_back_text = translate_and_back(text, sleep_time=0.1)
    if trans_back_text is not None:
        translated_text_list.append(trans_back_text)
    else:
        # retry limit
        print("Retry limited, break")
        break

print("Total translated: ", len(translated_text_list))

pickle.dump(translated_text_list, open("translated_text_list.pkl", "wb"))


0 / 7613
10 / 7613
20 / 7613
30 / 7613
40 / 7613
50 / 7613
60 / 7613
70 / 7613
80 / 7613
90 / 7613
Total translated:  100


In [20]:
#translated_text_list_loaded = pickle.load(open("translated_text_list.pkl", "rb"))
#translated_text_list_loaded